In [1]:
import glob
import re

INPUT_FOLDER = '../../../travelogues-corpus/18th_century/books'

def read_file(f):
  with open(f, 'r') as file:
    str = file.read()
    ascii_only = re.sub('[^A-Za-z0-9 ]+', '', str)
    return re.sub('\\s+', ' ', ascii_only)

filenames = [f for f in glob.glob(INPUT_FOLDER + '**/*.txt')]
texts = [ read_file(f) for f in filenames ]

f'Loaded {len(texts)} texts'

'Loaded 949 texts'

In [2]:
extra_stopwords = []

with open('stopwords.txt') as f:
  extra_stopwords = f.readlines()
  extra_stopwords = [ x.strip() for x in extra_stopwords ]

In [3]:
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('german')
stop_words.extend(extra_stopwords)
stop_words.sort()

[nltk_data] Downloading package stopwords to /home/simonr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import itertools
import matplotlib.pyplot as plt
import string

# We'll do stemming to avoid duplicate variants
porter = PorterStemmer()

def stem_message(text):
  words = word_tokenize(text)

  stemmed_words = []
  for word in words:
    stemmed_words.append(porter.stem(word))
    
  # Remove stopwords here
  clean_tokens = [w for w in stemmed_words if not w in stop_words and len(w) > 3] 
  return ' '.join(clean_tokens)

cleaned_texts = []
for text in texts:
  # Remove punctutation
  text = text.translate(str.maketrans('', '', string.punctuation))
  cleaned_texts.append(stem_message(text))

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+')
tf = vectorizer.fit_transform(cleaned_texts).toarray()

tf_feature_names = vectorizer.get_feature_names()

number_of_topics = 6
model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)
model.fit(tf)

/home/simonr/Software/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=6, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [8]:
import pandas as pd

def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

no_top_words = 20
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
display_topics(model, tf_feature_names, no_top_words)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights
0,kirch,12945.7,wert,7640.8,ieht,9282.0,schiff,15659.3,schiff,21239.3,kirch,12595.7
1,waffer,9263.4,bach,5247.5,teht,8192.7,reis,8978.7,inel,16368.9,gott,7639.9
2,brief,9073.6,ufer,5052.6,dorf,7813.5,insel,8801.7,reie,13142.0,laen,7277.7
3,schne,7707.1,gebirg,5051.0,tehen,7445.7,waffer,8115.8,laen,11475.5,wien,7099.1
4,reis,6677.0,gebrg,4844.1,kirch,7384.9,wind,6113.4,ollt,9779.7,drey,5252.4
5,stehen,6569.0,farb,4633.4,verchieden,7228.7,kste,5957.5,verchieden,9435.6,buch,4933.6
6,menschen,6337.0,zoll,4172.7,reie,7046.1,verschieden,5813.6,beonder,9313.8,heil,4543.6
7,besond,6315.8,dorf,4063.3,flecken,6495.4,drei,5034.9,wind,9306.6,herzog,4412.8
8,sieht,6225.7,drey,3296.3,meiten,6350.7,ufer,4867.6,menchen,8866.1,agen,4374.3
9,steht,5971.4,faden,3240.3,drei,6314.1,inseln,4843.2,konnten,8841.3,dabey,4343.1
